## Computations of the conditional MF and MI

This notebook implements algorithms 1 and 2 to compute:

    - The conditional MF of an attack s_n, given the attack unconditional probability (MF_LS) AND the conditional MF of its parent attacks and their join type (OR, AND)

    - The conditional MI of an attack s_n, given the attack unconditional probability (MF_LS) AND the conditional MF of its parent attacks and their join type (OR, AND)
    




In [57]:


import math, numpy as np, pandas as pd

def calculate_union_probability(events):
    '''
        input: Given an array of probabilities of events, 
        output: returns the resulting union of the probabilities of the events
    '''
    # Initialize the union probability as the probability of the first event
    union_prob = events[0]

    # Iterate through the remaining events
    for i in range(1, len(events)):
        event = events[i]

        # Calculate P(A ∩ B)
        intersection_prob = union_prob * event

        # Calculate P(A - B) and P(B - A)
        difference_prob_A = union_prob - intersection_prob
        difference_prob_B = event - intersection_prob

        # Update the union probability using the formula P(A ∪ B) = P(A - B) + P(A ∩ B) + P(B - A)
        union_prob = difference_prob_A + intersection_prob + difference_prob_B
        #print(union_prob, difference_prob_A, intersection_prob, difference_prob_B )
        

    return union_prob

def calculate_intersection_probability(events):
    '''
        input: Given an array of probabilities of events, 
        output: returns the resulting intersection of the probabilities of the independent events
    '''
    # Calculate the intersection of independent events
    intersection_prob = 1.0
    for event_prob in events:
        intersection_prob *= event_prob
    return intersection_prob

def calculate_mf_conditional_probability(attack_parents, mf_ls, join_type):
    '''
    attack_parents: a list containing the probabilities of the parents
    mf_ls : local static misuse Frequency of the child attack
    output: the conditional MF of an attack s_n, given the join type
    
    '''
    
    if join_type =='OR': 
        misuse_pa = calculate_union_probability(attack_parents)

    elif join_type =='AND': 
        misuse_pa = calculate_intersection_probability(attack_parents)
    else:
        print('ERROR ON JOIN TYPE; ',join_type )
    
    mf_conditional = mf_ls*misuse_pa
    
    return mf_conditional
   
    
def calculate_mi_conditional_probability(attack_parents, mi_ls, join_type):
    '''
    attack_parents: list containing the probabilities of the parents attack nodes
    mi_ls : local static misuse Impact of the child node attack
    
    Note: in the attack_parents list, it is possible to have negative probabilities. those are for attacks with destructive effects
    the signs will be adjusted in the function previous to calling this function

    output: the conditional MI of an attack s_n, given the join type
    
    '''
    if join_type =='OR': 
        mi_pa = calculate_union_probability(attack_parents)
        
    if join_type =='AND': 
        mi_pa = calculate_intersection_probability(attack_parents)
        
    mi_conditional =  mi_ls*(mi_ls + mi_pa)
    
    return mi_conditional


def preconditions_to_list(dataframe):
    '''
    gets the df of attack modules and changes the pre-conditions type to list 
    '''
    # Split the cell contents into a list based on the delimiter (comma or semicolon)
    delimiter = ','  # Change to ';' if you used a semicolon
    element = dataframe['pre-conditions']
    #print(element)
    number_list = [int(x.strip()) for x in element.split(delimiter)]
    #print(number_list)
    dataframe['pre-conditions'] = number_list
    
    return dataframe
    


In [78]:

file = pd.read_csv('mi_mf_attacks_computations_csv.csv', sep =';')

#print(file.head(5))

number_base = 4

print(  len( file.columns ) , len(file ), file.shape) 

out = pd.concat( [file.iloc[:number_base][:], file.iloc[number_base:][:].apply(preconditions_to_list, axis=1)], axis=0 )

#print(out.head(5))
# converting numbers in float
out['MF_LS_'] = out['MF_LS'].str.replace(',', '.').astype(float)
out['MI_LS_'] = out['MI_LS'].str.replace(',', '.').astype(float)
out['mu'] = out['mu'].str.replace(',', '.').astype(float)
out['beta'] = out['beta'].str.replace(',', '.').astype(float)
print(out.dtypes)

out['MI_LS'] = out['MI_LS_'] * out['beta']
out['MF_LS'] = out['MF_LS_'] * out['mu']


out.head(5)

8 52 (52, 8)
Attack number       int64
pre-conditions     object
join type          object
Attack             object
MF_LS              object
MI_LS              object
mu                float64
beta              float64
MF_LS_            float64
MI_LS_            float64
dtype: object


,Attack number,pre-conditions,join type,Attack,MF_LS,MI_LS,mu,beta,MF_LS_,MI_LS_
0,0,NaN,NaN,Topology discovery,0.724667,0.514417,0.8,0.75,0.905833,0.685889
1,1,NaN,NaN,limited authentication,0.724889,0.497917,0.8,0.75,0.906111,0.663889
2,2,NaN,NaN,attacker in communication range,0.685067,0.397015,0.8,0.75,0.856333,0.529353
3,3,NaN,NaN,user code executed,0.682133,0.397015,0.8,0.75,0.852667,0.529353
4,4,"[1, 25]",OR,Node Subversion integrity,0.686400,0.499480,0.8,0.75,0.858000,0.665973


In [79]:

# rounding the df values

base_attacks_0 = pd.DataFrame(data=out.iloc[:number_base][:]).drop(columns=['mu','beta','MF_LS_','MI_LS_'])
#print(base_attacks_0.head(5))

base_attacks_0['mf_conditional'] = round(base_attacks_0['MF_LS'],3)
base_attacks_0['mi_conditional'] = round(base_attacks_0['MI_LS'],3)

In [80]:
base_attacks_0

,Attack number,pre-conditions,join type,Attack,MF_LS,MI_LS,mf_conditional,mi_conditional
0,0,NaN,NaN,Topology discovery,0.724667,0.514417,0.725,0.514
1,1,NaN,NaN,limited authentication,0.724889,0.497917,0.725,0.498
2,2,NaN,NaN,attacker in communication range,0.685067,0.397015,0.685,0.397
3,3,NaN,NaN,user code executed,0.682133,0.397015,0.682,0.397


In [81]:


# for loop on the out preconditions to check if effective and compute conditional
base_attacks = base_attacks_0.copy()

while len(base_attacks) != len(out):
    for index in range(number_base,len(out)):
        
        # check if the attack has already been processed and is present in base_attacks
        if index not in base_attacks['Attack number'].to_list():
            #print(index)
            #print( out.iloc[index]['pre-conditions'] ) 
            pre_conditions =  set ( i for i in out.iloc[index]['pre-conditions'] ) # (abs(i) for i in out.iloc[index]['pre-conditions'] )
            existing_attacks = set(base_attacks['Attack number'].to_list())
            if {abs(x) for x in pre_conditions}.issubset(existing_attacks): # means we have everything to compute the attack mf and mi
                # get the join type on preconditions, mf and mi LS of the preconditions
                
                join_type = out.iloc[index]['join type']
                mf_ls = out.iloc[index]['MF_LS']
                mi_ls = out.iloc[index]['MI_LS']

                # get the mf and mi of the parents, consider the constructive and destructive effect
                pa_mi = []
                pa_mf = []
                for attack_id in pre_conditions : 
                    if attack_id <0:
                        
                        pa_mi.append(-base_attacks.loc[abs(attack_id)]['mi_conditional'])
                        pa_mf.append(base_attacks.loc[abs(attack_id)]['mf_conditional'])
                    else:                        
                        pa_mf.append(base_attacks.loc[abs(attack_id)]['mf_conditional'])
                        pa_mi.append(base_attacks.loc[abs(attack_id)]['mi_conditional'])
 

                mf_cond = calculate_mf_conditional_probability(pa_mf, mf_ls, join_type)
                mi_cond = calculate_mi_conditional_probability(pa_mi, mi_ls, join_type)

                row = [index, pre_conditions, join_type, out.iloc[index]['Attack'], mf_ls, mi_ls, mf_cond, mi_cond  ]

                #base_attacks = pd.concat([base_attacks, pd.DataFrame(row)], axis=0 ) 
                base_attacks.loc[index] = row

            else:
                #print('\n this else')
                #print(pre_conditions, existing_attacks)
                #print(base_attacks)
                continue
            







In [72]:
base_attacks

,Attack number,pre-conditions,join type,Attack,MF_LS,MI_LS,mf_conditional,mi_conditional
0,0,NaN,NaN,Topology discovery,0.724667,0.514417,0.725000,0.514000
1,1,NaN,NaN,limited authentication,0.724889,0.497917,0.725000,0.498000
2,2,NaN,NaN,attacker in communication range,0.685067,0.397015,0.685000,0.397000
3,3,NaN,NaN,user code executed,0.682133,0.397015,0.682000,0.397000
5,5,{2},OR,Jamming & DoS,0.666667,0.323833,0.456667,0.233430
10,10,"{0, 2}",AND,Data injection/ modification integrity,0.698400,0.541147,0.346843,0.403265
11,11,"{0, 2}",AND,Altering attack integrity,0.686400,0.333073,0.340883,0.178904
18,18,"{1, 2}",OR,Flooding 1,0.666667,0.323833,0.608917,0.330675
19,19,{0},OR,Physical attack,0.684667,0.386125,0.496383,0.347561
20,20,"{0, 3}",OR,Malware attack Availaibility,0.724133,0.397015,0.660808,0.438287
